In [23]:
import numpy as np
import pandas as pd
import time
from scipy.integrate import solve_ivp

In [24]:
# A function to calculate the physiological parameters of the model
# given the mass of the rat and the compartments of the model

def create_params(comp_names, w):
    all_comps = ["RoB","Heart", "Kidneys", "Brain", "Spleen", "Lungs", 
                 "Liver", "Uterus", "Bone", "Adipose", "Skin", "Muscles",
                 "GIT"]

    # Density of tissues/organs
    d_tissue = 1 #g/ml
    d_skeleton = 1.92 #g/ml
    d_adipose = 0.940 #g/ml

    Q_total =(1.54*w**0.75)*60 # Total Cardiac Output (ml/h)

    Total_Blood = 0.06*w+0.77 # Total blood volume (ml)

    fr_ad = 0.0199*w + 1.644 # w in g,  Brown et al.1997 p.420. This equation gives the  adipose % of body weight 

    #read data from excel
    fractions = pd.read_excel(r'Rat physiological parameters.xlsx')

    #Tissue weight fraction 
    Tissue_fractions = fractions.iloc[:,1]/100 # % of BW. Na values refers to the volume of the rest organs(RoB)
    Tissue_fractions.index = fractions.iloc[:,0] # replace the row names with the corresponding organ 
    Tissue_fractions[9] = fr_ad/100
    #Regional blood flow fraction
    Regional_flow_fractions = fractions.iloc[:,2]/100 # % of total cardiac output
    Regional_flow_fractions.index = fractions.iloc[:,0] # replace the row names with the corresponding organ
    #Capillary volume fractions (fractions of tissue volume)
    Capillary_fractions = fractions.iloc[:,3] # of tissue volume
    Capillary_fractions.index = fractions.iloc[:,0] # replace the row names with the corresponding organ

    W_tis = np.zeros(len(comp_names))
    V_tis = np.zeros(len(comp_names))
    V_cap = np.zeros(len(comp_names))
    Q = np.zeros(len(comp_names))

    # The following values were calculated by dividing the %ID/ g tissue with the %ID w/o free 48 from Table 2 of Kreyling et al. (2017)
    # Thus, they represent the average mass, in grams, of the respective tissues in each time group.
    liver_expw = np.mean([8.57, 8.92, 9.30, 8.61, 9.20])
    spleen_expw = np.mean([0.93, 0.75, 0.97, 0.68, 0.71])
    kidneys_expw = np.mean([2.27, 2.36, 2.44, 2.11, 2.26])
    lungs_expw = np.mean([1.87, 1.60, 1.80, 1.48, 1.31])
    heart_expw = np.mean([0.89, 1.00, 1.00, 1.00, 0.88])
    blood_expw = np.mean([16.52, 17.45, 15.33, 18.50, 18.00])
    carcass_expw = np.mean([206.00, 203.33, 184.00, 202.00, 203.75])
    skeleton_expw = np.mean([26.15, 27.50, 25.56, 25.79, 25.26])
    soft_tissues = np.mean([228.57, 253.85, 214.29, 225.93, 231.04])

    # Calculation of tissue weights  
    W_tis[1] = heart_expw
    W_tis[2] = kidneys_expw
    W_tis[4] = spleen_expw
    W_tis[5] = lungs_expw
    W_tis[6] = liver_expw
    W_tis[8] = skeleton_expw
    W_tis[12] = Tissue_fractions[12]*w

    for i in range(len(comp_names)):
        control = comp_names[i]
        if control == "NA":
            Regional_flow_fractions[i] = np.nan
            Capillary_fractions.iloc[i] = np.nan
        #Calculation of tissue volumes
        if i==8:
            V_tis[i] = W_tis[i]/d_skeleton
        elif i==9:
            V_tis[i] = W_tis[i]/d_adipose
        else:
            V_tis[i] = W_tis[i]/d_tissue 

        #Calculation of capillary volumes
        V_cap[i] = V_tis[i]*Capillary_fractions[i]

        #Calculation of regional blood flows
        Q[i] = Q_total*Regional_flow_fractions[i]


    # Calculations for "Soft tissue" compartment        
    W_tis[0] = w - W_tis[1:].sum() - Total_Blood 
    V_tis[0] = W_tis[0]/d_adipose
    Q[0] = 2*Q_total - np.nansum(Q[1:])
    V_cap[0] = V_tis[0]*Capillary_fractions[0]

    V_ven=0.64*Total_Blood
    V_art=0.15*Total_Blood
    Wm_ven=0.01*V_ven
    Wm_art=0.01*V_art
    
    V_blood=Total_Blood
    
    w_rob, w_ht, w_ki, w_spl, w_lu, w_li, w_bone, w_git = W_tis[[0,1,2,4,5,6,8,12]]
    V_tis_rob, V_tis_ht, V_tis_ki, V_tis_spl, V_tis_lu, V_tis_li, V_tis_bone, V_tis_git = V_tis[[0,1,2,4,5,6,8,12]]
    V_cap_rob, V_cap_ht, V_cap_ki, V_cap_spl, V_cap_lu, V_cap_li, V_cap_bone, V_cap_git = V_cap[[0,1,2,4,5,6,8,12]]    
    Q_rob, Q_ht, Q_ki, Q_spl, Q_lu, Q_li, Q_bone, Q_git = Q[[0,1,2,4,5,6,8,12]]
    
    
    
    return[Q_total, V_blood, V_ven, V_art,
           w_rob, w_ht, w_ki, w_spl, w_lu, w_li, w_bone, w_git,
           V_tis_rob, V_tis_ht, V_tis_ki, V_tis_spl, V_tis_lu, V_tis_li, V_tis_bone, V_tis_git,
           V_cap_rob, V_cap_ht, V_cap_ki, V_cap_spl, V_cap_lu, V_cap_li, V_cap_bone, V_cap_git,
           Q_rob, Q_ht, Q_ki, Q_spl, Q_lu, Q_li, Q_bone, Q_git
          ]
    
    

In [25]:
# A function to set up the initial conditions of the ODEs.
def create_inits(dose):
    M_ht=0; M_lu=0; M_li=0; M_spl=0; 
    M_ki=0; M_git=0; M_bone=0; M_rob=0;

    M_cap_ht=0; M_cap_lu=0; 
    M_cap_li=0; M_cap_spl=0; 
    M_cap_ki=0; M_cap_git=0; 
    M_cap_bone=0; M_cap_rob=0;

    M_lumen = 0;
    M_ven = dose; M_art=0
    M_feces=0; M_urine=0 
    
    return[M_ht, M_lu, M_li, M_spl, M_ki, M_git, M_bone, M_rob,
           M_cap_ht, M_cap_lu, M_cap_li, M_cap_spl, M_cap_ki, M_cap_git, M_cap_bone, M_cap_rob,
           M_lumen, M_ven, M_art, M_feces, M_urine
          ]
    

In [26]:
# The ODEs in a function 
def ode_func(t, m, params, x):
    M_ht, M_lu, M_li, M_spl, M_ki, M_git, M_bone, M_rob, \
    M_cap_ht, M_cap_lu, M_cap_li, M_cap_spl, M_cap_ki, \
    M_cap_git, M_cap_bone, M_cap_rob, \
    M_lumen, M_ven, M_art, M_feces, M_urine = m
        
    Q_total, V_blood, V_ven, V_art, \
    w_rob, w_ht, w_ki, w_spl, w_lu, w_li, w_bone, w_git, \
    V_tis_rob, V_tis_ht, V_tis_ki, V_tis_spl, V_tis_lu, V_tis_li, V_tis_bone, V_tis_git, \
    V_cap_rob, V_cap_ht, V_cap_ki, V_cap_spl, V_cap_lu, V_cap_li, V_cap_bone, V_cap_git, \
    Q_rob, Q_ht, Q_ki, Q_spl, Q_lu, Q_li, Q_bone, Q_git = params 
    
    P_ht,P_lu,P_li,P_spl,P_ki,P_git,P_bone,P_rob, \
    x_ht,x_lu,x_li,x_spl,x_ki,x_git,x_bone,x_rob,CLE_f,CLE_h = x 
    
    CLE_u = 0
    
    # Concentrations (mg of NPs)/(g of wet tissue)
    C_ht = M_ht/w_ht
    C_cap_ht = M_cap_ht/V_cap_ht
    C_lu = M_lu/w_lu
    C_cap_lu = M_cap_lu/V_cap_lu
    C_li = M_li/w_li
    C_cap_li = M_cap_li/V_cap_li
    C_spl = M_spl/w_spl
    C_cap_spl = M_cap_spl/V_cap_spl
    C_ki = M_ki/w_ki
    C_cap_ki = M_cap_ki/V_cap_ki
    C_git = M_git/w_git
    C_cap_git = M_cap_git/V_cap_git
    C_bone = M_bone/w_bone
    C_cap_bone = M_cap_bone/V_cap_bone
    C_rob = M_rob/w_rob
    C_cap_rob = M_cap_rob/V_cap_rob
    
    C_ven = M_ven/V_ven
    C_art = M_art/V_art
    
    # Heart
    dM_cap_ht = Q_ht*(C_art - C_cap_ht) - x_ht*Q_ht*(C_cap_ht - C_ht/P_ht)
    dM_ht = x_ht*Q_ht*(C_cap_ht - C_ht/P_ht) 

    # Lungs
    dM_cap_lu = Q_total*(C_ven - C_cap_lu) - x_lu*Q_total*(C_cap_lu - C_lu/P_lu)
    dM_lu = x_lu*Q_total*(C_cap_lu - C_lu/P_lu)

    # Liver 
    dM_cap_li = Q_li*(C_art - C_cap_li) + Q_spl*(C_cap_spl - C_cap_li) + Q_git*(C_cap_git - C_cap_li) - \
                x_li*(Q_li)*(C_cap_li - C_li/P_li)
    dM_li = x_li*Q_li*(C_cap_li - C_li/P_li) - CLE_h*M_li

    # Spleen
    dM_cap_spl = Q_spl*(C_art - C_cap_spl) - x_spl*Q_spl*(C_cap_spl - C_spl/P_spl)
    dM_spl = x_spl*Q_spl*(C_cap_spl - C_spl/P_spl) 

    # Kidneys
    dM_cap_ki = Q_ki*(C_art - C_cap_ki) - x_ki*Q_ki*(C_cap_ki - C_ki/P_ki)- CLE_u*M_cap_ki
    dM_ki = x_ki*Q_ki*(C_cap_ki - C_ki/P_ki) 

    # GIT - Gastrointestinal Tract
    dM_cap_git = Q_git*(C_art - C_cap_git) - x_git*Q_git*(C_cap_git - C_git/P_git)
    dM_git = x_git*Q_git*(C_cap_git - C_git/P_git) 
    dM_lumen = CLE_h*M_li - CLE_f *M_lumen 

    # Bone
    dM_cap_bone = Q_bone*(C_art - C_cap_bone) - x_bone*Q_bone*(C_cap_bone - C_bone/P_bone)
    dM_bone = x_bone*Q_bone*(C_cap_bone - C_bone/P_bone) 


    # RoB - Rest of Body
    dM_cap_rob = Q_rob*(C_art - C_cap_rob) - x_rob*Q_rob*(C_cap_rob - C_rob/P_rob)
    dM_rob = x_rob*Q_rob*(C_cap_rob - C_rob/P_rob) 

    # Urine
    dM_urine = CLE_u*M_cap_ki

    # Feces
    dM_feces = CLE_f*M_lumen

    # Venous Blood
    dM_ven = Q_ht*C_cap_ht + (Q_li + Q_spl+Q_git)*C_cap_li + Q_ki*C_cap_ki + \
             Q_bone*C_cap_bone + Q_rob*C_cap_rob - Q_total*C_ven

    # Arterial Blood
    dM_art = Q_total*C_cap_lu - Q_total*C_art
    
    Blood_total = M_ven + M_art + M_cap_ht + M_cap_lu +M_cap_li+M_cap_spl+ \
                   M_cap_ki+ M_cap_git+M_cap_bone+M_cap_rob
    Blood = Blood_total/(V_blood)
    
    C_soft = (M_git+M_lumen+M_rob)/(w_git + w_rob)
    
    return[dM_ht, dM_lu, dM_li, dM_spl, dM_ki, dM_git, 
           dM_bone, dM_rob, dM_cap_ht, dM_cap_lu, 
           dM_cap_li, dM_cap_spl, dM_cap_ki, dM_cap_git, 
           dM_cap_bone, dM_cap_rob,
           dM_lumen, dM_ven, dM_art, 
           dM_feces, dM_urine]
#            Blood,
#            C_ht, C_lu, C_li, C_spl, 
#            C_ki, C_bone, C_soft,
#            M_feces]

In [27]:
# A function to create x values
def create_x(x):
    P1,P2,P3,P4,P5,P6,P7,P8,X1,X2,X3,X4,X5,X6,X7,X8,CLE_f,CLE_h = x
    return[P1,P2,P3,P4,P5,P6,P7,P8,X1,X2,X3,X4,X5,X6,X7,X8,CLE_f,CLE_h]

In [28]:
# Input
comp_names = ["RoB","Heart", "Kidneys", "NA", "Spleen",
              "Lungs", "Liver", "NA", "Bone","NA", "NA", "NA", "GIT"]
w = 263
params = create_params(comp_names, w)

dose = 18.15
inits = create_inits(dose)

x_values = [1.584382e+01, 9.739224e+00, 1.524937e+01, 8.996928e+00, 1.284654e+01, 1.781151e+01,
            1.172908e+01, 1.469810e+01, 2.071897e-02, 2.097445e-02, 2.026367e-02, 4.340532e-02,
            1.857050e-02, 4.358142e-02, 2.086843e-02, 3.243307e-02, 1.546329e-01, 9.864152e-05]
x = create_x(x_values)
#print(x)

t_span = (0, 28*24)
#t_span = (0,9)

t_eval = range(0, 28*24+1, 1)
#t_eval = range(0,10,1)


C:\Users\vassi\AppData\Local\Temp\ipykernel_17832\1515324740.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Capillary_fractions.iloc[i] = np.nan


In [19]:
# LSODA 
start_time = time.time()
solution_LSODA = solve_ivp(fun=ode_func, t_span=t_span, method='LSODA', t_eval=t_eval, 
                     atol = 1e-05, rtol = 1e-05,
                     y0=inits, args=[params, x])
finish_time = time.time()
LSODA_time = (finish_time - start_time)
LSODA_time

0.03690195083618164

In [8]:
# BDF 
start_time = time.time()
solution_BDF = solve_ivp(fun=ode_func, t_span=t_span, method='BDF', t_eval=t_eval, 
                     atol = 1e-5, rtol = 1e-5,
                     y0=inits, args=[params, x])
finish_time = time.time()
BDF_time = (finish_time - start_time)
BDF_time

0.08280014991760254

In [30]:
[params, x]

[[6034.465137122706,
  16.55,
  10.592,
  2.4825,
  198.71499999999997,
  0.9540000000000001,
  2.288,
  0.8080000000000002,
  1.612,
  8.920000000000002,
  26.052,
  7.101000000000001,
  211.39893617021275,
  0.9540000000000001,
  2.288,
  0.8080000000000002,
  1.612,
  8.920000000000002,
  13.56875,
  7.101000000000001,
  8.45595744680851,
  0.24804000000000004,
  0.36607999999999996,
  0.17776000000000003,
  0.5803200000000001,
  1.8732000000000002,
  0.54275,
  0.3550500000000001,
  2183.269486610994,
  295.6887917190126,
  850.8595843343015,
  73.62047467289702,
  6034.465137122706,
  1049.9969338593507,
  736.2047467289701,
  844.825119197179],
 [15.84382,
  9.739224,
  15.24937,
  8.996928,
  12.84654,
  17.81151,
  11.72908,
  14.6981,
  0.02071897,
  0.02097445,
  0.02026367,
  0.04340532,
  0.0185705,
  0.04358142,
  0.02086843,
  0.03243307,
  0.1546329,
  9.864152e-05]]

In [20]:
column_names = ['M_ht', 'M_lu', 'M_li', 'M_spl', 'M_ki', 'M_git', 'M_bone', 'M_rob',
                'M_cap_ht', 'M_cap_lu', 'M_cap_li', 'M_cap_spl', 'M_cap_ki', 'M_cap_git', 'M_cap_bone', 'M_cap_rob',
                'M_lumen', 'M_ven', 'M_art', 'M_feces', 'M_urine']

solution_df = pd.DataFrame(solution_LSODA.y.T, columns = column_names)
solution_df

,M_ht,M_lu,M_li,M_spl,M_ki,M_git,M_bone,M_rob,M_cap_ht,M_cap_lu,...,M_cap_spl,M_cap_ki,M_cap_git,M_cap_bone,M_cap_rob,M_lumen,M_ven,M_art,M_feces,M_urine
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.150000,0.000000,0.000000,0.0
1,0.521114,0.492166,2.149004,0.261326,1.225701,3.338863,1.688491,7.840429,0.006230,0.014441,...,0.004520,0.009232,0.008867,0.013310,0.205612,0.000157,0.262112,0.061809,0.000011,0.0
2,0.436141,0.233348,2.185388,0.214769,0.930658,3.062031,1.865572,8.853977,0.003648,0.008363,...,0.002675,0.005392,0.005263,0.007731,0.118935,0.000334,0.152540,0.035779,0.000049,0.0
3,0.359074,0.199371,2.135646,0.174163,0.709039,2.726585,1.971961,9.554105,0.003161,0.007261,...,0.002306,0.004658,0.004577,0.006725,0.103387,0.000484,0.132484,0.031063,0.000112,0.0
4,0.299940,0.178991,2.063507,0.143822,0.560564,2.427100,2.051270,10.136363,0.002837,0.006535,...,0.002065,0.004175,0.004116,0.006062,0.093188,0.000606,0.119242,0.027956,0.000197,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,0.076381,0.079334,0.686946,0.036735,0.148531,0.639143,1.544223,14.761588,0.001253,0.002933,...,0.000898,0.001850,0.001794,0.002743,0.042730,0.000438,0.053524,0.012545,0.046945,0.0
669,0.076381,0.079334,0.686943,0.036735,0.148530,0.639141,1.544217,14.761533,0.001253,0.002932,...,0.000898,0.001850,0.001794,0.002743,0.042730,0.000438,0.053524,0.012545,0.047012,0.0
670,0.076380,0.079334,0.686941,0.036735,0.148530,0.639138,1.544211,14.761477,0.001253,0.002932,...,0.000898,0.001850,0.001794,0.002743,0.042730,0.000438,0.053524,0.012545,0.047080,0.0
671,0.076380,0.079334,0.686938,0.036735,0.148529,0.639136,1.544206,14.761422,0.001253,0.002932,...,0.000898,0.001850,0.001794,0.002743,0.042730,0.000438,0.053524,0.012545,0.047148,0.0
